# Reduced Model with autoReduce attempt 13
In this notebook, we will compare the full model to the reduced model 1: without complexes. We don't expect a difference but we see one. <br>
7.21.2020

In [1]:
from libsbml import *
import sys
import numpy as np
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

from sympy import Symbol,sympify

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

## Copy function 
---

In [2]:
# def sbml_to_ode(filename):
#     '''A function that takes in an SBML file and returns x,f,P,params_values.
#     x is a list of species written as Sympy objects
#     f is a list of functions written as Sympy objects
#     P is a list of parameters written as Sympy objects
#     params_values is a list of parameter values, in the same order as P'''

#     # Get the sbml file, check for errors, and perform conversions
#     doc = readSBMLFromFile(filename)
#     if doc.getNumErrors(LIBSBML_SEV_FATAL):
#         print('Encountered serious errors while reading file')
#         print(doc.getErrorLog().toString())
#         sys.exit(1)
    
#     doc.getErrorLog().clearLog()
    
#     # Convert local params to global params
#     props = ConversionProperties()
#     props.addOption("promoteLocalParameters", True)
  
#     if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
#         print('The document could not be converted')
#         print(doc.getErrorLog().toString())
    
#     # Expand initial assignments
#     props = ConversionProperties()
#     props.addOption("expandInitialAssignments", True)
  
#     if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
#         print('The document could not be converted')
#         print(doc.getErrorLog().toString())
    
#     # Expand functions definitions
#     props = ConversionProperties()
#     props.addOption("expandFunctionDefinitions", True)
  
#     if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
#         print('The document could not be converted')
#         print(doc.getErrorLog().toString())
    
#     # Get model and define importnat lists, dictionaries
#     mod = doc.getModel()
#     x = []
#     P = []
#     params_values = []
#     reactions = {}
 
#     # Append species symbol to 'x'
#     for i in range(mod.getNumSpecies()): 
#         species = mod.getSpecies(i)
#         x.append(Symbol(species.getId()))
    
#     # Append parameter symbol to 'P' and parameter values to 'params_values'
#     for i in range(mod.getNumParameters()): 
#         params = mod.getParameter(i)
#         params_values.append(params.getValue())
#         P.append(Symbol(params.getId()))

    
    
    
#     # Get kinetic formula for each reaction, store in dictionary 'reactions'
#     for i in range(mod.getNumReactions()): 
#         reaction = mod.getReaction(i)
#         kinetics = reaction.getKineticLaw()
#         reactions[reaction.getId()] = sympify(kinetics.getFormula())

#     # Define f
#     f = [0] * len(x)
    
#     # Loop to define functions in 'f'
#     for i in range(mod.getNumReactions()): 
#         reaction = mod.getReaction(i)
#         # subtract reactant kinetic formula
#         for j in range(reaction.getNumReactants()):
#             ref = reaction.getReactant(j)
#             species = sympify(mod.getSpecies(ref.getSpecies()).getId())
#             curr_index = x.index(species)
#             # Check stoichiometry
#             if ref.getStoichiometry() == 1.0:  
#                 f[curr_index] += -reactions[reaction.getId()]
#             else:
#                 f[curr_index] += -reactions[reaction.getId()] * ref.getStoichiometry()
#         # add product kinetic formula
#         for j in range(reaction.getNumProducts()):
#             ref = reaction.getProduct(j)
#             species = sympify(mod.getSpecies(ref.getSpecies()).getId())
#             curr_index = x.index(species)
#             # Check stoichiometry
#             if ref.getStoichiometry() == 1.0: 
#                 f[curr_index] += +reactions[reaction.getId()]
#             else:
#                 f[curr_index] += +reactions[reaction.getId()] * ref.getStoichiometry()

#     return x,f,P,params_values

In [3]:
def sbml_to_ode(filename):
    '''A function that takes in an SBML file and returns x,f,P,params_values.
    x is a list of species written as Sympy objects
    f is a list of functions written as Sympy objects
    P is a list of parameters written as Sympy objects
    params_values is a list of parameter values, in the same order as P
    x_init is a list of initial conditions, in the same order as x'''

    # Get the sbml file, check for errors, and perform conversions
    doc = readSBMLFromFile(filename)
    if doc.getNumErrors(LIBSBML_SEV_FATAL):
        print('Encountered serious errors while reading file')
        print(doc.getErrorLog().toString())
        sys.exit(1)
    
    doc.getErrorLog().clearLog()
    
    # Convert local params to global params
    props = ConversionProperties()
    props.addOption("promoteLocalParameters", True)
  
    if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
        print('The document could not be converted')
        print(doc.getErrorLog().toString())
    
    # Expand initial assignments
    props = ConversionProperties()
    props.addOption("expandInitialAssignments", True)
  
    if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
        print('The document could not be converted')
        print(doc.getErrorLog().toString())
    
    # Expand functions definitions
    props = ConversionProperties()
    props.addOption("expandFunctionDefinitions", True)
  
    if doc.convert(props) != LIBSBML_OPERATION_SUCCESS: 
        print('The document could not be converted')
        print(doc.getErrorLog().toString())
    
    # Get model and define importnat lists, dictionaries
    mod = doc.getModel()
    x = []
    x_init = []
    P = []
    params_values = []
    reactions = {}
 
    # Append species symbol to 'x' and append initial amount/concentration to x_init
    # x[i] corresponds to x_init[i]
    for i in range(mod.getNumSpecies()): 
        species = mod.getSpecies(i)
        x.append(Symbol(species.getId()))
        if species.isSetInitialConcentration():
            x_init.append(species.getInitialConcentration())
        elif species.isSetInitialAmount():
            x_init.append(species.getInitialAmount())
        else:
            x_init.append(0)

    # Append parameter symbol to 'P' and parameter values to 'params_values'
    for i in range(mod.getNumParameters()): 
        params = mod.getParameter(i)
        params_values.append(params.getValue())
        P.append(Symbol(params.getId()))

    
    
    
    # Get kinetic formula for each reaction, store in dictionary 'reactions'
    for i in range(mod.getNumReactions()): 
        reaction = mod.getReaction(i)
        kinetics = reaction.getKineticLaw()
        reactions[reaction.getId()] = sympify(kinetics.getFormula())

    # Define f
    f = [0] * len(x)
    
    # Loop to define functions in 'f'
    for i in range(mod.getNumReactions()): 
        reaction = mod.getReaction(i)
        # subtract reactant kinetic formula
        for j in range(reaction.getNumReactants()):
            ref = reaction.getReactant(j)
            species = sympify(mod.getSpecies(ref.getSpecies()).getId())
            curr_index = x.index(species)
            # Check stoichiometry
            if ref.getStoichiometry() == 1.0:  
                f[curr_index] += -reactions[reaction.getId()]
            else:
                f[curr_index] += -reactions[reaction.getId()] * ref.getStoichiometry()
        # add product kinetic formula
        for j in range(reaction.getNumProducts()):
            ref = reaction.getProduct(j)
            species = sympify(mod.getSpecies(ref.getSpecies()).getId())
            curr_index = x.index(species)
            # Check stoichiometry
            if ref.getStoichiometry() == 1.0: 
                f[curr_index] += +reactions[reaction.getId()]
            else:
                f[curr_index] += +reactions[reaction.getId()] * ref.getStoichiometry()


    return x,f,P,params_values,x_init

## Plot Full Model 
---

In [4]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode(filename)

In [5]:
nouts = 1
n = len(x)
C = np.zeros((nouts, n), dtype = int)
C[0][62] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,1000)

#x_init = np.zeros(n)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

In [6]:
p_reg = bokeh.plotting.figure(width = 475, height = 350, title = 'Species to be removed in reduced model',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
p_reg2 = bokeh.plotting.figure(width = 475, height = 350, title = 'Full Model from SBML',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
import colorcet
from bokeh.layouts import row
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
p_reg2.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p_reg2.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p_reg2.line(timepoints_ode, sol[:,62], line_width = 2,legend_label = 'isobutanol', color = colors[1])

species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,66,67]
for i,spec in enumerate(species_to_remove):
    p_reg.line(timepoints_ode, sol[:,spec], line_width = 2, color = colors[i])
# p_reg.line(timepoints_ode, sol[:,43], line_width = 2,legend_label = 'random1', color = colors[4])
# p_reg.line(timepoints_ode, sol[:,14], line_width = 2,legend_label = 'random2', color = colors[5])
# p_reg.line(timepoints_ode, sol[:,5], line_width = 2,legend_label = 'random3', color = colors[6])

#p_reg.legend.click_policy = 'hide'
bokeh.io.show(row(p_reg,p_reg2))

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/models/plots.py:764: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


Probably different because all above complexes are considered constant (not really being solved for) <br>
Red and pink are not reaching ss until 40 hrs <br>
Bottom ones are going to ss earlier (5/10 tp) <br>
Should remove species that quickly approach steady state (time scale separation) <br>
Can remove things that aren't complexes

<font color = 'red'>
    RETAIN RED AND PINK IN REDUCED MODEL
    </font>
   <br>


**Figure out which ones are the red and pink ones**

In [10]:
p_look = bokeh.plotting.figure(width = 475, height = 350, title = 'Find red and pink',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
species = [66, 67]

for i, spec in enumerate(species):
    p_look.line(timepoints_ode, sol[:,spec], line_width = 2, color = colors[i], legend_label = str(species[i]))

bokeh.io.show(p_look)

## Plot Reduced Model 1: Remove all complexes
---

Define important things again.

In [46]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode(filename)

nouts = 1
n = len(x)
# Make ATP main output
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,100)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

Define list of species to remove

In [47]:
species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,66,67]

Define `update_timescale`, a list to pass into `solve_timescale_separation`. It should consist of the species that we want to retain

In [48]:
timescale_list = []
for i in species_to_remove:
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)    
#update_timescale

In [49]:
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)

In [53]:
aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
print('the ATP error is:',sys_reduce.get_error_metric(aa))

attempting to retain : [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65]
Successful time-scale separation solution obtained.
the ATP error is: 1466.3386384090306


In [51]:
sys_ode = get_ODE(aa, timepoints_ode)
sol = sys_ode.solve_system()

In [54]:
p_reg = bokeh.plotting.figure(width = 475, height = 350, title = 'RED MOD 1',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p_reg.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p_reg.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p_reg.line(timepoints_ode, sol[:,34], line_width = 2,legend_label = 'isobutanol', color = colors[1])
bokeh.io.show(p_reg)

This model is good for steady state dynamics (not for ATP transient dynamics)

In [58]:
aa.f[1]

enzyme_atpase*metabolite_atp*r75_1_k*r75_k/(r75_1_k + r77_k) - enzyme_atpase*metabolite_atp*r75_k + enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(r0_1_k + r2_k) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k + enzyme_pfk*metabolite_atp*molecule_f6p*r10_1_k*r10_k/(r10_1_k + r12_k) - enzyme_pfk*metabolite_atp*molecule_f6p*r10_k - enzyme_pgk*metabolite_atp*r33_1_k*pow(molecule_3pg, 2) + enzyme_pgk*(metabolite_adp*molecule_13bpg*r30_k*r32_k + metabolite_atp*r30_1_k*r33_1_k*pow(molecule_3pg, 2) + metabolite_atp*r32_k*r33_1_k*pow(molecule_3pg, 2))/(r30_1_k + r32_k) - 2.0*enzyme_pyk*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + 2.0*enzyme_pyk*(r45_1_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + r45_k*r47_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2) + r47_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2))/(r45_1_k + r47_k)

Now, plot function with `scipy.odeint()`

In [27]:
from scipy.integrate import odeint 

# Figure out a way to equate param name to param value
generated_code = bytearray()
for i, param_name in enumerate(P):
    generated_code.extend('{0} = {1}\n'.format(param_name, params_values[i]).encode('latin-1'))
exec(generated_code)
 
# Define rhs func
def ode_fun(x, t): # REMOVE 
    x0 = enzyme_hex = x[0]
    x1 = metabolite_atp = x[1]
    x2 = molecule_glucose = x[2]
    x3 = molecule_g6p = x[3]
    x4 = metabolite_adp = x[4]
    x7 = enzyme_pgi = x[5]
    x8 = molecule_f6p = x[6]
    x11 = enzyme_pfk = x[7]
    x12 = molecule_f16p = x[8]
    x15 = enzyme_ald_tpi = x[9]
    x16 = molecule_g3p = x[10]
    x19 = enzyme_gapN = x[11]
    x20 = metabolite_nadp = x[12]
    x21 = molecule_3pg = x[13]
    x22 = metabolite_nadph = x[14]
    x25 = enzyme_mGapDH = x[15]
    x26 = metabolite_pi = x[16]
    x27 = molecule_13bpg = x[17]
    x30 = enzyme_pgk = x[18]
    x33 = enzyme_pgm = x[19]
    x34 = molecule_2pg = x[20]
    x37 = enzyme_eno = x[21]
    x38 = molecule_pep = x[22]
    x41 = enzyme_pyk = x[23]
    x42 = molecule_pyruvate = x[24]
    x45 = enzyme_alsS = x[25]
    x46 = molecule_acetolac = x[26]
    x49 = enzyme_IlvC = x[27]
    x50 = molecule_23dih3mebut = x[28]
    x53 = enzyme_IlvD = x[29]
    x54 = molecule_3me2oxo = x[30]
    x57 = enzyme_kivD = x[31]
    x58 = molecule_isobutanal = x[32]
    x61 = enzyme_yahk = x[33]
    x62 = molecule_isobutanol = x[34]
    x65 = enzyme_atpase = x[35]
      
 
    return np.array([-enzyme_hex*metabolite_adp*molecule_g6p*r3_1_k + enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(r0_1_k + r2_k) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k + enzyme_hex*(metabolite_adp*molecule_g6p*r0_1_k*r3_1_k + metabolite_adp*molecule_g6p*r2_k*r3_1_k + metabolite_atp*molecule_glucose*r0_k*r2_k)/(r0_1_k + r2_k),
 enzyme_atpase*metabolite_atp*r75_1_k*r75_k/(r75_1_k + r77_k) - enzyme_atpase*metabolite_atp*r75_k + enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(r0_1_k + r2_k) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k + enzyme_pfk*metabolite_atp*molecule_f6p*r10_1_k*r10_k/(r10_1_k + r12_k) - enzyme_pfk*metabolite_atp*molecule_f6p*r10_k - enzyme_pgk*metabolite_atp*r33_1_k*pow(molecule_3pg, 2) + enzyme_pgk*(metabolite_adp*molecule_13bpg*r30_k*r32_k + metabolite_atp*r30_1_k*r33_1_k*pow(molecule_3pg, 2) + metabolite_atp*r32_k*r33_1_k*pow(molecule_3pg, 2))/(r30_1_k + r32_k) - 2.0*enzyme_pyk*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + 2.0*enzyme_pyk*(r45_1_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + r45_k*r47_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2) + r47_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2))/(r45_1_k + r47_k),
 enzyme_hex*metabolite_atp*molecule_glucose*r0_1_k*r0_k/(r0_1_k + r2_k) - enzyme_hex*metabolite_atp*molecule_glucose*r0_k,
 -enzyme_hex*metabolite_adp*molecule_g6p*r3_1_k + enzyme_hex*(metabolite_adp*molecule_g6p*r0_1_k*r3_1_k + metabolite_adp*molecule_g6p*r2_k*r3_1_k + metabolite_atp*molecule_glucose*r0_k*r2_k)/(r0_1_k + r2_k) + enzyme_pgi*molecule_g6p*r5_1_k*r5_k/(r5_1_k + r7_k) - enzyme_pgi*molecule_g6p*r5_k,
 -enzyme_atpase*metabolite_adp*metabolite_pi*r78_1_k + enzyme_atpase*(metabolite_adp*metabolite_pi*r75_1_k*r78_1_k + metabolite_adp*metabolite_pi*r77_k*r78_1_k + metabolite_atp*r75_k*r77_k)/(r75_1_k + r77_k) - enzyme_hex*metabolite_adp*molecule_g6p*r3_1_k + enzyme_hex*(metabolite_adp*molecule_g6p*r0_1_k*r3_1_k + metabolite_adp*molecule_g6p*r2_k*r3_1_k + metabolite_atp*molecule_glucose*r0_k*r2_k)/(r0_1_k + r2_k) - enzyme_pfk*metabolite_adp*molecule_f16p*r13_1_k + enzyme_pfk*(metabolite_adp*molecule_f16p*r10_1_k*r13_1_k + metabolite_adp*molecule_f16p*r12_k*r13_1_k + metabolite_atp*molecule_f6p*r10_k*r12_k)/(r10_1_k + r12_k) + enzyme_pgk*metabolite_adp*molecule_13bpg*r30_1_k*r30_k/(r30_1_k + r32_k) - enzyme_pgk*metabolite_adp*molecule_13bpg*r30_k + 2.0*enzyme_pyk*r45_1_k*r45_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2)/(r45_1_k + r47_k) - 2.0*enzyme_pyk*r45_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2),
 -enzyme_pgi*molecule_f6p*r8_1_k + enzyme_pgi*molecule_g6p*r5_1_k*r5_k/(r5_1_k + r7_k) - enzyme_pgi*molecule_g6p*r5_k + enzyme_pgi*(molecule_f6p*r5_1_k*r8_1_k + molecule_f6p*r7_k*r8_1_k + molecule_g6p*r5_k*r7_k)/(r5_1_k + r7_k),
 enzyme_pfk*metabolite_atp*molecule_f6p*r10_1_k*r10_k/(r10_1_k + r12_k) - enzyme_pfk*metabolite_atp*molecule_f6p*r10_k - enzyme_pgi*molecule_f6p*r8_1_k + enzyme_pgi*(molecule_f6p*r5_1_k*r8_1_k + molecule_f6p*r7_k*r8_1_k + molecule_g6p*r5_k*r7_k)/(r5_1_k + r7_k),
 -enzyme_pfk*metabolite_adp*molecule_f16p*r13_1_k + enzyme_pfk*metabolite_atp*molecule_f6p*r10_1_k*r10_k/(r10_1_k + r12_k) - enzyme_pfk*metabolite_atp*molecule_f6p*r10_k + enzyme_pfk*(metabolite_adp*molecule_f16p*r10_1_k*r13_1_k + metabolite_adp*molecule_f16p*r12_k*r13_1_k + metabolite_atp*molecule_f6p*r10_k*r12_k)/(r10_1_k + r12_k),
 enzyme_ald_tpi*molecule_f16p*r15_1_k*r15_k/(r15_1_k + r17_k) - enzyme_ald_tpi*molecule_f16p*r15_k - enzyme_pfk*metabolite_adp*molecule_f16p*r13_1_k + enzyme_pfk*(metabolite_adp*molecule_f16p*r10_1_k*r13_1_k + metabolite_adp*molecule_f16p*r12_k*r13_1_k + metabolite_atp*molecule_f6p*r10_k*r12_k)/(r10_1_k + r12_k),
 enzyme_ald_tpi*molecule_f16p*r15_1_k*r15_k/(r15_1_k + r17_k) - enzyme_ald_tpi*molecule_f16p*r15_k - enzyme_ald_tpi*r18_1_k*pow(molecule_g3p, 2) + enzyme_ald_tpi*(molecule_f16p*r15_k*r17_k + r15_1_k*r18_1_k*pow(molecule_g3p, 2) + r17_k*r18_1_k*pow(molecule_g3p, 2))/(r15_1_k + r17_k),
 -2.0*enzyme_ald_tpi*r18_1_k*pow(molecule_g3p, 2) + 2.0*enzyme_ald_tpi*(molecule_f16p*r15_k*r17_k + r15_1_k*r18_1_k*pow(molecule_g3p, 2) + r17_k*r18_1_k*pow(molecule_g3p, 2))/(r15_1_k + r17_k) + 2.0*enzyme_gapN*r20_1_k*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r20_1_k + r22_k) - 2.0*enzyme_gapN*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + 2.0*enzyme_mGapDH*metabolite_pi*r25_1_k*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r25_1_k + r27_k) - 2.0*enzyme_mGapDH*metabolite_pi*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2),
 enzyme_gapN*r20_1_k*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r20_1_k + r22_k) - enzyme_gapN*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) - enzyme_gapN*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2) + enzyme_gapN*(r20_1_k*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2) + r20_k*r22_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + r22_k*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2))/(r20_1_k + r22_k),
 -enzyme_IlvC*metabolite_nadp*molecule_23dih3mebut*r58_1_k + enzyme_IlvC*(metabolite_nadp*molecule_23dih3mebut*r55_1_k*r58_1_k + metabolite_nadp*molecule_23dih3mebut*r57_k*r58_1_k + metabolite_nadph*molecule_acetolac*r55_k*r57_k)/(r55_1_k + r57_k) + 2.0*enzyme_gapN*r20_1_k*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r20_1_k + r22_k) - 2.0*enzyme_gapN*r20_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + 2.0*enzyme_mGapDH*metabolite_pi*r25_1_k*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r25_1_k + r27_k) - 2.0*enzyme_mGapDH*metabolite_pi*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) - enzyme_yahk*metabolite_nadp*molecule_isobutanol*r73_1_k + enzyme_yahk*(metabolite_nadp*molecule_isobutanol*r70_1_k*r73_1_k + metabolite_nadp*molecule_isobutanol*r72_k*r73_1_k + metabolite_nadph*molecule_isobutanal*r70_k*r72_k)/(r70_1_k + r72_k),
 -2.0*enzyme_gapN*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2) + 2.0*enzyme_gapN*(r20_1_k*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2) + r20_k*r22_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + r22_k*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2))/(r20_1_k + r22_k) - 2.0*enzyme_pgk*metabolite_atp*r33_1_k*pow(molecule_3pg, 2) + 2.0*enzyme_pgk*(metabolite_adp*molecule_13bpg*r30_k*r32_k + metabolite_atp*r30_1_k*r33_1_k*pow(molecule_3pg, 2) + metabolite_atp*r32_k*r33_1_k*pow(molecule_3pg, 2))/(r30_1_k + r32_k) + 2.0*enzyme_pgm*r35_1_k*r35_k*pow(molecule_3pg, 2)/(r35_1_k + r37_k) - 2.0*enzyme_pgm*r35_k*pow(molecule_3pg, 2),
 enzyme_IlvC*metabolite_nadph*molecule_acetolac*r55_1_k*r55_k/(r55_1_k + r57_k) - enzyme_IlvC*metabolite_nadph*molecule_acetolac*r55_k - 2.0*enzyme_gapN*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2) + 2.0*enzyme_gapN*(r20_1_k*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2) + r20_k*r22_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + r22_k*r23_1_k*pow(metabolite_nadph, 2)*pow(molecule_3pg, 2))/(r20_1_k + r22_k) - 2.0*enzyme_mGapDH*molecule_13bpg*r28_1_k*pow(metabolite_nadph, 2) + 2.0*enzyme_mGapDH*(metabolite_pi*r25_k*r27_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + molecule_13bpg*r25_1_k*r28_1_k*pow(metabolite_nadph, 2) + molecule_13bpg*r27_k*r28_1_k*pow(metabolite_nadph, 2))/(r25_1_k + r27_k) + enzyme_yahk*metabolite_nadph*molecule_isobutanal*r70_1_k*r70_k/(r70_1_k + r72_k) - enzyme_yahk*metabolite_nadph*molecule_isobutanal*r70_k,
 enzyme_mGapDH*metabolite_pi*r25_1_k*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r25_1_k + r27_k) - enzyme_mGapDH*metabolite_pi*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) - enzyme_mGapDH*molecule_13bpg*r28_1_k*pow(metabolite_nadph, 2) + enzyme_mGapDH*(metabolite_pi*r25_k*r27_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + molecule_13bpg*r25_1_k*r28_1_k*pow(metabolite_nadph, 2) + molecule_13bpg*r27_k*r28_1_k*pow(metabolite_nadph, 2))/(r25_1_k + r27_k),
 -enzyme_atpase*metabolite_adp*metabolite_pi*r78_1_k + enzyme_atpase*(metabolite_adp*metabolite_pi*r75_1_k*r78_1_k + metabolite_adp*metabolite_pi*r77_k*r78_1_k + metabolite_atp*r75_k*r77_k)/(r75_1_k + r77_k) + enzyme_mGapDH*metabolite_pi*r25_1_k*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2)/(r25_1_k + r27_k) - enzyme_mGapDH*metabolite_pi*r25_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2),
 -enzyme_mGapDH*molecule_13bpg*r28_1_k*pow(metabolite_nadph, 2) + enzyme_mGapDH*(metabolite_pi*r25_k*r27_k*pow(metabolite_nadp, 2)*pow(molecule_g3p, 2) + molecule_13bpg*r25_1_k*r28_1_k*pow(metabolite_nadph, 2) + molecule_13bpg*r27_k*r28_1_k*pow(metabolite_nadph, 2))/(r25_1_k + r27_k) + enzyme_pgk*metabolite_adp*molecule_13bpg*r30_1_k*r30_k/(r30_1_k + r32_k) - enzyme_pgk*metabolite_adp*molecule_13bpg*r30_k,
 enzyme_pgk*metabolite_adp*molecule_13bpg*r30_1_k*r30_k/(r30_1_k + r32_k) - enzyme_pgk*metabolite_adp*molecule_13bpg*r30_k - enzyme_pgk*metabolite_atp*r33_1_k*pow(molecule_3pg, 2) + enzyme_pgk*(metabolite_adp*molecule_13bpg*r30_k*r32_k + metabolite_atp*r30_1_k*r33_1_k*pow(molecule_3pg, 2) + metabolite_atp*r32_k*r33_1_k*pow(molecule_3pg, 2))/(r30_1_k + r32_k),
 enzyme_pgm*r35_1_k*r35_k*pow(molecule_3pg, 2)/(r35_1_k + r37_k) - enzyme_pgm*r35_k*pow(molecule_3pg, 2) - enzyme_pgm*r38_1_k*pow(molecule_2pg, 2) + enzyme_pgm*(r35_1_k*r38_1_k*pow(molecule_2pg, 2) + r35_k*r37_k*pow(molecule_3pg, 2) + r37_k*r38_1_k*pow(molecule_2pg, 2))/(r35_1_k + r37_k),
 2.0*enzyme_eno*r40_1_k*r40_k*pow(molecule_2pg, 2)/(r40_1_k + r42_k) - 2.0*enzyme_eno*r40_k*pow(molecule_2pg, 2) - 2.0*enzyme_pgm*r38_1_k*pow(molecule_2pg, 2) + 2.0*enzyme_pgm*(r35_1_k*r38_1_k*pow(molecule_2pg, 2) + r35_k*r37_k*pow(molecule_3pg, 2) + r37_k*r38_1_k*pow(molecule_2pg, 2))/(r35_1_k + r37_k),
 enzyme_eno*r40_1_k*r40_k*pow(molecule_2pg, 2)/(r40_1_k + r42_k) - enzyme_eno*r40_k*pow(molecule_2pg, 2) - enzyme_eno*r43_1_k*pow(molecule_pep, 2) + enzyme_eno*(r40_1_k*r43_1_k*pow(molecule_pep, 2) + r40_k*r42_k*pow(molecule_2pg, 2) + r42_k*r43_1_k*pow(molecule_pep, 2))/(r40_1_k + r42_k),
 -2.0*enzyme_eno*r43_1_k*pow(molecule_pep, 2) + 2.0*enzyme_eno*(r40_1_k*r43_1_k*pow(molecule_pep, 2) + r40_k*r42_k*pow(molecule_2pg, 2) + r42_k*r43_1_k*pow(molecule_pep, 2))/(r40_1_k + r42_k) + 2.0*enzyme_pyk*r45_1_k*r45_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2)/(r45_1_k + r47_k) - 2.0*enzyme_pyk*r45_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2),
 enzyme_pyk*r45_1_k*r45_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2)/(r45_1_k + r47_k) - enzyme_pyk*r45_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2) - enzyme_pyk*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + enzyme_pyk*(r45_1_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + r45_k*r47_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2) + r47_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2))/(r45_1_k + r47_k),
 2.0*enzyme_alsS*r50_1_k*r50_k*pow(molecule_pyruvate, 2)/(r50_1_k + r52_k) - 2.0*enzyme_alsS*r50_k*pow(molecule_pyruvate, 2) - 2.0*enzyme_pyk*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + 2.0*enzyme_pyk*(r45_1_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2) + r45_k*r47_k*pow(metabolite_adp, 2)*pow(molecule_pep, 2) + r47_k*r48_1_k*pow(metabolite_atp, 2)*pow(molecule_pyruvate, 2))/(r45_1_k + r47_k),
 -enzyme_alsS*molecule_acetolac*r53_1_k + enzyme_alsS*r50_1_k*r50_k*pow(molecule_pyruvate, 2)/(r50_1_k + r52_k) - enzyme_alsS*r50_k*pow(molecule_pyruvate, 2) + enzyme_alsS*(molecule_acetolac*r50_1_k*r53_1_k + molecule_acetolac*r52_k*r53_1_k + r50_k*r52_k*pow(molecule_pyruvate, 2))/(r50_1_k + r52_k),
 enzyme_IlvC*metabolite_nadph*molecule_acetolac*r55_1_k*r55_k/(r55_1_k + r57_k) - enzyme_IlvC*metabolite_nadph*molecule_acetolac*r55_k - enzyme_alsS*molecule_acetolac*r53_1_k + enzyme_alsS*(molecule_acetolac*r50_1_k*r53_1_k + molecule_acetolac*r52_k*r53_1_k + r50_k*r52_k*pow(molecule_pyruvate, 2))/(r50_1_k + r52_k),
 -enzyme_IlvC*metabolite_nadp*molecule_23dih3mebut*r58_1_k + enzyme_IlvC*metabolite_nadph*molecule_acetolac*r55_1_k*r55_k/(r55_1_k + r57_k) - enzyme_IlvC*metabolite_nadph*molecule_acetolac*r55_k + enzyme_IlvC*(metabolite_nadp*molecule_23dih3mebut*r55_1_k*r58_1_k + metabolite_nadp*molecule_23dih3mebut*r57_k*r58_1_k + metabolite_nadph*molecule_acetolac*r55_k*r57_k)/(r55_1_k + r57_k),
 -enzyme_IlvC*metabolite_nadp*molecule_23dih3mebut*r58_1_k + enzyme_IlvC*(metabolite_nadp*molecule_23dih3mebut*r55_1_k*r58_1_k + metabolite_nadp*molecule_23dih3mebut*r57_k*r58_1_k + metabolite_nadph*molecule_acetolac*r55_k*r57_k)/(r55_1_k + r57_k) + enzyme_IlvD*molecule_23dih3mebut*r60_1_k*r60_k/(r60_1_k + r62_k) - enzyme_IlvD*molecule_23dih3mebut*r60_k,
 enzyme_IlvD*molecule_23dih3mebut*r60_1_k*r60_k/(r60_1_k + r62_k) - enzyme_IlvD*molecule_23dih3mebut*r60_k - enzyme_IlvD*molecule_3me2oxo*r63_1_k + enzyme_IlvD*(molecule_23dih3mebut*r60_k*r62_k + molecule_3me2oxo*r60_1_k*r63_1_k + molecule_3me2oxo*r62_k*r63_1_k)/(r60_1_k + r62_k),
 -enzyme_IlvD*molecule_3me2oxo*r63_1_k + enzyme_IlvD*(molecule_23dih3mebut*r60_k*r62_k + molecule_3me2oxo*r60_1_k*r63_1_k + molecule_3me2oxo*r62_k*r63_1_k)/(r60_1_k + r62_k) + enzyme_kivD*molecule_3me2oxo*r65_1_k*r65_k/(r65_1_k + r67_k) - enzyme_kivD*molecule_3me2oxo*r65_k,
 enzyme_kivD*molecule_3me2oxo*r65_1_k*r65_k/(r65_1_k + r67_k) - enzyme_kivD*molecule_3me2oxo*r65_k - enzyme_kivD*molecule_isobutanal*r68_1_k + enzyme_kivD*(molecule_3me2oxo*r65_k*r67_k + molecule_isobutanal*r65_1_k*r68_1_k + molecule_isobutanal*r67_k*r68_1_k)/(r65_1_k + r67_k),
 -enzyme_kivD*molecule_isobutanal*r68_1_k + enzyme_kivD*(molecule_3me2oxo*r65_k*r67_k + molecule_isobutanal*r65_1_k*r68_1_k + molecule_isobutanal*r67_k*r68_1_k)/(r65_1_k + r67_k) + enzyme_yahk*metabolite_nadph*molecule_isobutanal*r70_1_k*r70_k/(r70_1_k + r72_k) - enzyme_yahk*metabolite_nadph*molecule_isobutanal*r70_k,
 -enzyme_yahk*metabolite_nadp*molecule_isobutanol*r73_1_k + enzyme_yahk*metabolite_nadph*molecule_isobutanal*r70_1_k*r70_k/(r70_1_k + r72_k) - enzyme_yahk*metabolite_nadph*molecule_isobutanal*r70_k + enzyme_yahk*(metabolite_nadp*molecule_isobutanol*r70_1_k*r73_1_k + metabolite_nadp*molecule_isobutanol*r72_k*r73_1_k + metabolite_nadph*molecule_isobutanal*r70_k*r72_k)/(r70_1_k + r72_k),
 -enzyme_yahk*metabolite_nadp*molecule_isobutanol*r73_1_k + enzyme_yahk*(metabolite_nadp*molecule_isobutanol*r70_1_k*r73_1_k + metabolite_nadp*molecule_isobutanol*r72_k*r73_1_k + metabolite_nadph*molecule_isobutanal*r70_k*r72_k)/(r70_1_k + r72_k),
 -enzyme_atpase*metabolite_adp*metabolite_pi*r78_1_k + enzyme_atpase*metabolite_atp*r75_1_k*r75_k/(r75_1_k + r77_k) - enzyme_atpase*metabolite_atp*r75_k + enzyme_atpase*(metabolite_adp*metabolite_pi*r75_1_k*r78_1_k + metabolite_adp*metabolite_pi*r77_k*r78_1_k + metabolite_atp*r75_k*r77_k)/(r75_1_k + r77_k)])

# Perform simulations
time = np.linspace(0, 75, 1000)
yinit = x_init

y = odeint(ode_fun, yinit, time)

p2 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 1: Remove All Complexes',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
p2.line(time, y[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p2.line(time, y[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p2.line(time, y[:,34], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p2.legend.click_policy = 'hide'
bokeh.io.show(p2)

RuntimeError: The size of the array returned by func (36) does not match the size of y0 (68).

Check x_init

In [14]:
# #From red mod
# array([ 0.15, 30.  , 30.  ,  0.  ,  0.  ,  0.15,  0.  ,  0.15,  0.  ,
#         0.15,  0.  ,  0.15, 30.  ,  0.  ,  0.  ,  0.15, 30.  ,  0.  ,
#         0.15,  0.15,  0.  ,  0.15,  0.  ,  0.15,  0.  ,  0.15,  0.  ,
#         0.15,  0.  ,  0.15,  0.  ,  0.15,  0.  ,  0.15,  0.  ,  2.  ])

# # From normal
# [ 0.2 30.  30.   0.   0.   0.2  0.   0.2  0.   0.2  0.   0.2  0.   0.
#   0.   0.2  0.   0.   0.   0.2 30.   0.   0.   0.   0.   0.2 30.   0.
#   0.   0.   0.2  0.   0.   0.2  0.   0.   0.   0.2  0.   0.   0.   0.2
#   0.   0.   0.   0.2  0.   0.   0.   0.2  0.   0.   0.   0.2  0.   0.
#   0.   0.2  0.   0.   0.   0.2  0.   0.   0.   2.   0.   0. ]

In [15]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,auto_reduce,sympy,libsbml,sys,jupyterlab

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
auto_reduce 0.1
sympy 1.5.1
libsbml 5.18.0
sys 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
jupyterlab 1.2.6
